In [2]:
import osmnx as ox
import networkx as nx
import pandas as pd
import os

In [3]:
os.chdir("..")
os.getcwd()

'/home/martin/Desktop/10-Academy/week-8/Causal-Inference'

In [4]:
completed_orders = pd.read_csv('/home/martin/Desktop/10-Academy/week-8/Causal-Inference/data/cleaned_completed_orders.csv')

In [ ]:
4	392001	6.6010417,3.2766339	6.4501069,3.3916154	2021-07-01 09:30:59	2021-07-01 09:34:36	3.6166666666666663	9	1	7	3	0	Rain	false	6.6010417	3.2766339	6.4501069	3.3916154	21.048262052555348	0.0269434998112587

In [6]:
# Define the start and end locations in latlng
start_latlng = (6.6010417, -3.2766339)
end_latlng = (6.4501069, -3.3916154)

# Location where you want to find your route
place = 'Lagos, Nigeria'

# Mode of travel ('drive', 'bike', 'walk')
mode = 'drive'

# Optimizer for finding the shortest path ('length', 'time')
optimizer = 'time'

# Create graph from OSM within the boundaries of the specified place
graph = ox.graph_from_place(place, network_type=mode)

# Find the nearest node to the start location
orig_node = ox.distance.nearest_nodes(graph, start_latlng[1], start_latlng[0])

# Find the nearest node to the end location
dest_node = ox.distance.nearest_nodes(graph, end_latlng[1], end_latlng[0])

# Find the shortest path
shortest_route = nx.shortest_path(graph, orig_node, dest_node, weight=optimizer)



In [ ]:

# Visualize the route
fig, ax = ox.plot_graph_route(graph, shortest_route, route_linewidth=4, origin_dest_node_size=50, node_size=None, show=False)
ax.set_title(f'Shortest {mode} Route from {start_latlng} to {end_latlng}')

# Assuming completed_orders is your DataFrame with 'Trip Distance' and 'Trip Start Time'/'Trip End Time' columns
def calculate_speed(row):
    distance = row['Trip Distance']
    start_time = pd.to_datetime(row['Trip Start Time'])
    end_time = pd.to_datetime(row['Trip End Time'])
    time_diff = end_time - start_time
    if time_diff.total_seconds() == 0:
        return 0  # Default value for trips with zero duration
    speed = distance / time_diff.total_seconds() * 3600  # Assuming distance is in km and converting speed to km/h
    return speed

# Calculate speeds for all rows in the DataFrame
completed_orders['Speed'] = completed_orders.apply(lambda row: calculate_speed(row), axis=1)

# Optionally, analyze the speeds for different modes of transportation
speed_by_mode = completed_orders.groupby('Mode').mean()['Speed']
print(speed_by_mode)

In [ ]:
shortest_route_map = ox.plot_route_folium(graph, shortest_route)
shortest_route_map

In [ ]:
shortest_route_map = ox.plot_route_folium(graph, shortest_route, 
                                          tiles='openstreetmap')
shortest_route_map

In [ ]:
import foliumfolium.TileLayer('openstreetmap').add_to(shortest_route_map)
folium.TileLayer('Stamen Terrain').add_to(shortest_route_map)
folium.TileLayer('Stamen Toner').add_to(shortest_route_map)
folium.TileLayer('Stamen Water Color').add_to(shortest_route_map)
folium.TileLayer('cartodbpositron').add_to(shortest_route_map)
folium.TileLayer('cartodbdark_matter').add_to(shortest_route_map)folium.LayerControl().add_to(shortest_route_map)shortest_route_map